### $\bullet\;Import\;Libraries$

In [ ]:
%pip install torchmetrics
%pip install torchview

In [ ]:
import torch
import torch.nn             as nn
import torch.optim          as optim
import torch.nn.functional  as F
import torch.backends.cudnn as cudnn
import torchvision
import matplotlib.pyplot    as plt
import numpy                as np
import pandas               as pd
import os
import pickle

from torch.utils.data       import TensorDataset, DataLoader, random_split
from torchvision            import transforms
from torchvision.datasets   import CIFAR10
from torchmetrics           import Accuracy
from tqdm                   import tqdm
from torch.utils.data       import Dataset
from PIL                    import Image
from math                   import sqrt

from torchview import draw_graph
from IPython.display import Image, display

plt.style.use('grayscale')

### $\bullet\;CIFAR\;Dataset$

#### $Load\;Dataset$

In [ ]:
dataset = CIFAR10(root='./cifar10', train=True, download=True, transform=transforms.ToTensor())

loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False, num_workers=2)
data_iter = iter(loader)
images, _ = next(data_iter) 

mean = images.mean(dim=[0, 2, 3])
std = images.std(dim=[0, 2, 3])

print("Mean per channel :", mean)
print("Std per channel  :", std)

In [ ]:
transform   = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])])

train_set   = CIFAR10(root='./cifar10', train=True , download=True, transform=transform)
test_set    = CIFAR10(root='./cifar10', train=False, download=True, transform=transform)

In [ ]:
len(train_set)

#### $Train\;set-Test\;set\;Classes$

In [ ]:
pd.DataFrame(train_set.class_to_idx.items(), columns=['Class Name', 'Class Index']).style.hide()

In [ ]:
pd.DataFrame(train_set.class_to_idx.items(), columns=['Class Name', 'Class Index']).style.hide()

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val    = 0
        self.avg    = 0
        self.sum    = 0
        self.count  = 0

    def update(self, val, n=1):
        self.val    = val
        self.sum    += val * n
        self.count  += n
        self.avg    = self.sum / self.count

### $\bullet\;$ CIFAR10 DataLoader, Visualization

In [ ]:
# DataLoader
trainloader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False, num_workers=4)

def denormalize(img, mean, std):
    mean = torch.tensor(mean).view(3, 1, 1)
    std = torch.tensor(std).view(3, 1, 1)
    return img * std + mean

# Show one batch of images
dataiter = iter(trainloader)
images, labels = next(dataiter)
img_grid = torchvision.utils.make_grid(images)
img_grid = denormalize(img_grid, mean, std)
plt.imshow(img_grid.permute(1, 2, 0).clamp(0, 1))
plt.axis('off')
plt.show()

### $\bullet\;$ CNN Model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 7 layers total: 6 conv + final classifier
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            # Block 2
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            # Block 3
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.classifier = nn.Linear(256 * 4 * 4, 10)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = CNN()
batch_size = 2
size_model = 0
for param in model.parameters():
    if param.data.is_floating_point():
        size_model += param.numel() * torch.finfo(param.data.dtype).bits
    else:
        size_model += param.numel() * torch.iinfo(param.data.dtype).bits
print(f"model size: {size_model} bit | {size_model / 8e6:.2f} MB")

model_graph = draw_graph(model, input_size=(1, 3, 32, 32), device='meta')
display(model_graph.visual_graph)

In [ ]:
def print_layer_outputs(model, input_size):
    x = torch.randn(input_size)
    print(f"Input: {x.shape}")
    count = 0
    for i, layer in enumerate(model.features):
        x = layer(x)
        if layer.__class__.__name__ in ['Conv2d']:
            count += 1
        print(f"Layer {count} ({layer.__class__.__name__}): {x.shape}")
    x = x.view(x.size(0), -1)
    print(f"Flatten: {x.shape}")
    x = model.classifier(x)
    print(f"Classifier Output: {x.shape}")

model = CNN().to('cpu')
print_layer_outputs(model, (1, 3, 32, 32))

In [ ]:
from torchinfo import summary
summary(model, input_size=(1, 3, 32, 32))

In [ ]:
size_model = 0
for param in model.parameters():
    if param.data.is_floating_point():
        size_model += param.numel()
    else:
        size_model += param.numel()

### $\bullet\;$ Train

#### loss, optimizer, scheduler

using CrossEntropyLoss

using Adam with lr=0.001, weight_decay=1e-4

In [ ]:
# Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)

# loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15)

In [ ]:
# Training loop with loss tracking
epochs = 15
train_accs, test_accs, train_losses, test_losses = [], [], [], []
for epoch in range(1, epochs+1):
    model.train()
    correct, total = 0, 0
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(trainloader, 1):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        _, preds = outputs.max(1)
        total += targets.size(0)
        correct += preds.eq(targets).sum().item()
        running_loss += loss.item() * inputs.size(0)
    train_accs.append(100.*correct/total)
    train_losses.append(running_loss / total)

    model.eval()
    correct, total = 0, 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, preds = outputs.max(1)
            total += targets.size(0)
            correct += preds.eq(targets).sum().item()
            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)
    test_acc = 100. * correct / total
    test_losses.append(running_loss / total)
    test_accs.append(test_acc)
    scheduler.step()
    print(f"Epoch {epoch:2d}: Train Acc = {train_accs[-1]:.2f}%, Test Acc = {test_acc:.2f}%")

In [ ]:
# Plot accuracy and loss curves
eps = range(1, epochs+1)
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(eps, train_accs, label='Train Acc')
plt.plot(eps, test_accs, label='Test Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Accuracy Curve')

plt.subplot(1,2,2)
plt.plot(eps, train_losses, label='Train Loss')
plt.plot(eps, test_losses, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')

plt.tight_layout()
plt.show()

# Phase 2

In [ ]:
import copy
import torch.nn.utils.prune as prune

baseline_model = copy.deepcopy(model)

In [ ]:
def test(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return 100.*correct/total

## Q1-بررسی هرس یکباره

ابتدا از روش کاملا تصادفی استفاده می کنیم. در این روش بدون توجه به هیچ چیزی شروع به هرس می کنیم و برایمان تفاوتی بین وزن ها و یال ها وجود ندارد.

In [ ]:
import matplotlib.pyplot as plt

pruned_models = []
prune_amounts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
accuracies = []

rand_model = dict()

for amount in prune_amounts:
    model_copy = copy.deepcopy(baseline_model)
    for module in model_copy.modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.random_unstructured(module, name='weight', amount=amount)
    pruned_models.append(model_copy)
    acc = test(model_copy)
    accuracies.append(acc) 
    rand_model[amount] = model_copy
    print(f'Random unstructured prune {amount*100:.0f}%, acc: {acc:.2f}%')

plt.figure(figsize=(10, 6))
plt.plot([amount * 100 for amount in prune_amounts], accuracies, marker='o', linestyle='-', color='b', label='Accuracy')
plt.xlabel('Prune Amount (%)')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs Prune Amount in random prune')
plt.grid(True)
plt.legend()
plt.show()

همانطور که انتظار داشتیم این روش اصلا مناسب نیست و از همان ابتدا دقت مدل را نصف می کند

حال از l1_unstructured استفاده می کنیم که در این روش وزن هایی که قدر مطلق آنها به صفر نزندیک تر است حذف می شود. در مقابل آن l2 رو داریم که به جای قدر مطلق از توان ۲ 
استفاده می کند

In [ ]:
pruned_models = []
prune_amounts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
accuracies = []

l1_uns_model = dict()

for amount in prune_amounts:
    model_copy = copy.deepcopy(baseline_model)
    for module in model_copy.modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
    pruned_models.append(model_copy)
    acc = test(model_copy)
    accuracies.append(acc) 
    l1_uns_model[amount] = model_copy
    print(f'l1 unstructured prune {amount*100:.0f}%, acc: {acc:.2f}%')

plt.figure(figsize=(10, 6))
plt.plot([amount * 100 for amount in prune_amounts], accuracies, marker='o', linestyle='-', color='b', label='Accuracy')
plt.xlabel('Prune Amount (%)')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs Prune Amount in l1 prune')
plt.grid(True)
plt.legend()
plt.show()

همانطور که در نمودار مشخص است هرس تا ۲۰ درصد تقریبا بی تاثیر است ولی باقی موارد بیش از ۱ درصد تاثیر دارند که مناسب نیست و همچنین افت دقت شدیدی بعد از حذف ۶۰ درصد شامل هستیم که دقت نصف شده

باید توجه کرد که در این روش ما با پیمایش روی شبکه در هر لایه به صورت جداگانه حرص انجام می دهیم. می شود با تجمیع وزن ها نیز این کار را انجام داد. از آنجایی که روش اول تصادفی است در نتیجه آن تفاوتی ایجاد نمی شود اما ممکن است در روش دوم تفاوت ایجاد کند که در ادامه بررسی می کنیم..

In [ ]:
pruned_models = []
prune_amounts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
accuracies = []

l1_uns_glo_model = dict()

parameters_to_prune = []
for module in baseline_model.modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        parameters_to_prune.append((module, 'weight'))

for amount in prune_amounts:
    model_copy = copy.deepcopy(baseline_model)
    parameters = []
    for module in model_copy.modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            parameters.append((module, 'weight'))

    prune.global_unstructured(
        parameters,
        pruning_method=prune.L1Unstructured,
        amount=amount,
    )
    
    pruned_models.append(model_copy)
    acc = test(model_copy)
    accuracies.append(acc) 
    l1_uns_glo_model[amount] = model_copy
    print(f'l1-global unstructured prune {amount*100:.0f}%, acc: {acc:.2f}%')

plt.figure(figsize=(10, 6))
plt.plot([amount * 100 for amount in prune_amounts], accuracies, marker='o', linestyle='-', color='b', label='Accuracy')
plt.xlabel('Prune Amount (%)')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs Prune Amount in l1-global prune')
plt.grid(True)
plt.legend()
plt.show()

همانطور که در نمودار مشخص است با این کار تقریبا تا ۴۰ درصد حرص قابل قبول است پس با این کار تحمل مدل در حرص ۲ برابر شد

## Q2- هرس مرحله به مرحله

In [ ]:
def train(epochs):
    for epoch in range(1, epochs + 1):
        model.train()
        correct, total = 0, 0
        running_loss = 0.0
        for i, (inputs, targets) in enumerate(trainloader, 1):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            _, preds = outputs.max(1)
            total += targets.size(0)
            correct += preds.eq(targets).sum().item()
            running_loss += loss.item() * inputs.size(0)

        train_accs.append(100. * correct / total)
        train_losses.append(running_loss / total)

        model.eval()
        correct, total = 0, 0
        running_loss = 0.0
        with torch.no_grad():
            for inputs, targets in testloader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                _, preds = outputs.max(1)
                total += targets.size(0)
                correct += preds.eq(targets).sum().item()
                loss = criterion(outputs, targets)
                running_loss += loss.item() * inputs.size(0)

        test_acc = 100. * correct / total
        test_accs.append(test_acc)
        test_losses.append(running_loss / total)
        scheduler.step()

        print(f"Epoch {epoch}/{epochs} - Train Acc: {train_accs[-1]:.2f}%, Test Acc: {test_acc:.2f}%")


In [ ]:
start_prune = 0.2
end_prune = 0.8
step_count = int((end_prune - start_prune) / 0.1) + 1
fine_tune_epochs = 4

model = copy.deepcopy(baseline_model)
parameters_to_prune = [
    (m, 'weight') for m in model.modules()
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear)
]

accuracies = []
prune_levels = []

l1_uns_glob_step = dict()

current_pruned = 0.0
target_pruned_list = [round(p, 2) for p in list(torch.arange(start_prune, end_prune + 0.001, 0.1).tolist())]

for target in target_pruned_list:
    remaining = 1.0 - current_pruned
    step_amount = (target - current_pruned) / remaining  

    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=step_amount
    )
    current_pruned = target

    # loss, optimizer, scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=fine_tune_epochs)

    train(fine_tune_epochs)

    acc = test(model)
    accuracies.append(acc)
    prune_levels.append(target * 100)
    l1_uns_glob_step[target] = copy.deepcopy(model)
    print(f'Pruned to {target*100:.0f}%, Accuracy: {acc:.2f}%')

plt.figure(figsize=(10, 6))
plt.plot(prune_levels, accuracies, marker='o', linestyle='-', color='b', label='Accuracy')
plt.xlabel('Prune Amount (%)')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs Prune Amount in L1-Global Iterative Prune')
plt.grid(True)
plt.legend()
plt.show()

با این کار دقت مدل تا ۵۰ درصد خوب است و ختی از ۴۰ درصد حالت قبلی بهتر است و همچنین دقت تمامی حالات بالا تر است مثلا در هرس ۸۰ درصد دقت ۶۳ درصد می شود در مقابل ۲۴ درصد

## Q3- هرس ساختارمند

In [ ]:
start_prune = 0.2
end_prune = 0.8
step_count = int((end_prune - start_prune) / 0.1) + 1
fine_tune_epochs = 4

model = copy.deepcopy(baseline_model)

accuracies = []
prune_levels = []

l2_struc_step_model = dict()

current_pruned = 0.0
target_pruned_list = [round(p, 2) for p in list(torch.arange(start_prune, end_prune + 0.001, 0.1).tolist())]

for target in target_pruned_list:
    remaining = 1.0 - current_pruned
    step_amount = (target - current_pruned) / remaining 

    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=step_amount, n=2, dim=0) 

    current_pruned = target

    # loss, optimizer, scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=fine_tune_epochs)

    train(fine_tune_epochs)

    acc = test(model)
    accuracies.append(acc)
    prune_levels.append(target * 100)
    l2_struc_step_model[target] = copy.deepcopy(model)
    print(f'L2 Structured Pruned to {target*100:.0f}%, Accuracy: {acc:.2f}%')

plt.figure(figsize=(10, 6))
plt.plot(prune_levels, accuracies, marker='o', linestyle='-', color='g', label='L2 Structured Accuracy')
plt.xlabel('Prune Amount (%)')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs Prune Amount in L2-Structured Iterative Prune')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
start_prune = 0.2
end_prune = 0.8
step_count = int((end_prune - start_prune) / 0.1) + 1
fine_tune_epochs = 4

model = copy.deepcopy(baseline_model)

accuracies = []
prune_levels = []

l1_struc_step_model = dict()

current_pruned = 0.0
target_pruned_list = [round(p, 2) for p in list(torch.arange(start_prune, end_prune + 0.001, 0.1).tolist())]

for target in target_pruned_list:
    remaining = 1.0 - current_pruned
    step_amount = (target - current_pruned) / remaining 

    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=step_amount, n=1, dim=0) 

    current_pruned = target

    # loss, optimizer, scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=fine_tune_epochs)

    train(fine_tune_epochs)

    acc = test(model)
    accuracies.append(acc)
    prune_levels.append(target * 100)
    l1_struc_step_model[target] = copy.deepcopy(model)
    print(f'L2 Structured Pruned to {target*100:.0f}%, Accuracy: {acc:.2f}%')

plt.figure(figsize=(10, 6))
plt.plot(prune_levels, accuracies, marker='o', linestyle='-', color='g', label='L1 Structured Accuracy')
plt.xlabel('Prune Amount (%)')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs Prune Amount in L2-Structured Iterative Prune')
plt.grid(True)
plt.legend()
plt.show()

# Phase 3

## Adding file in block (for using in kaggle)

In [ ]:
import copy
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

class Quantize(torch.nn.Module):
    def __init__(self, n_bits, n_frac, sign=True):
        super(Quantize, self).__init__()
        self.n_bits = n_bits
        self.n_frac = n_frac
        i = self.n_bits - self.n_frac
        if sign:
            self.max = float(2 ** (i - 1) - 2 ** (-self.n_frac))
            self.min = float(-2 ** (i - 1))
        else:
            self.max = float(2 ** (i) - 2 ** (-self.n_frac))
            self.min = 0.0

    def forward(self, x):
        if torch.onnx.is_in_onnx_export():
            return x
        else:
            n = float(2 ** self.n_frac)
            xx = torch.floor(x * n) / n
            clipped = torch.clip(xx, self.min, self.max)
            return clipped

class Round(Function):
    @staticmethod
    def forward(self, input):
        sign = torch.sign(input)
        output = sign * torch.floor(torch.abs(input) + 0.5)
        return output

    @staticmethod
    def backward(self, grad_output):
        grad_input = grad_output.clone()
        return grad_input

class ALSQPlus(Function):
    @staticmethod
    def forward(ctx, weight, alpha, beta, g, Qn, Qp):
        ctx.save_for_backward(weight, alpha, beta)
        ctx.other = g, Qn, Qp
        w_q = Round.apply(torch.div((weight - beta), alpha).clamp(Qn, Qp))
        w_q = w_q * alpha + beta
        return w_q

    @staticmethod
    def backward(ctx, grad_weight):
        weight, alpha, beta = ctx.saved_tensors
        g, Qn, Qp = ctx.other
        q_w = (weight - beta) / alpha
        smaller = (q_w < Qn).float()
        bigger = (q_w > Qp).float()
        between = 1.0 - smaller -bigger
        grad_alpha = ((smaller * Qn + bigger * Qp +
            between * Round.apply(q_w) - between * q_w)*grad_weight * g).sum().unsqueeze(dim=0)
        grad_beta = ((smaller + bigger) * grad_weight * g).sum().unsqueeze(dim=0)
        grad_weight = between * grad_weight
        return grad_weight, grad_alpha, grad_beta, None, None, None


class WLSQPlus(Function):
    @staticmethod
    def forward(ctx, weight, alpha, g, Qn, Qp, per_channel):
        ctx.save_for_backward(weight, alpha)
        ctx.other = g, Qn, Qp, per_channel
        if per_channel:
            sizes = weight.size()
            # weight = weight.contiguous().view(weight.size()[0], -1)
            weight = weight.view(weight.size()[0], -1)
            weight = torch.transpose(weight, 0, 1)
            alpha = torch.broadcast_to(alpha, weight.size())
            wq = Round.apply(torch.div(weight, alpha).clamp(Qn, Qp))
            w_q = wq * alpha
            w_q = torch.transpose(w_q, 0, 1)
            # w_q = w_q.contiguous().view(sizes)
            w_q = w_q.view(sizes)
            wq = torch.transpose(wq, 0, 1).view(sizes)
        else:
            wq = Round.apply(torch.div(weight, alpha).clamp(Qn, Qp))
            w_q = wq * alpha
        return w_q, wq.detach()

    @staticmethod
    def backward(ctx, grad_weight, gwq):
        weight, alpha = ctx.saved_tensors
        g, Qn, Qp, per_channel = ctx.other
        if per_channel:
            sizes = weight.size()
            # weight = weight.contiguous().view(weight.size()[0], -1)
            weight = weight.view(weight.size()[0], -1)
            weight = torch.transpose(weight, 0, 1)
            alpha = torch.broadcast_to(alpha, weight.size())
            q_w = weight / alpha
            q_w = torch.transpose(q_w, 0, 1)
            # q_w = q_w.contiguous().view(sizes)
            q_w = q_w.view(sizes)
        else:
            q_w = weight / alpha
        smaller = (q_w < Qn).float()
        bigger = (q_w > Qp).float()
        between = 1.0 - smaller -bigger
        if per_channel:
            grad_alpha = ((smaller * Qn + bigger * Qp +
                between * Round.apply(q_w) - between * q_w)*grad_weight * g)
            # grad_alpha = grad_alpha.contiguous().view(grad_alpha.size()[0], -1).sum(dim=1)
            grad_alpha = grad_alpha.view(grad_alpha.size()[0], -1).sum(dim=1)
        else:
            grad_alpha = ((smaller * Qn + bigger * Qp +
                between * Round.apply(q_w) - between * q_w)*grad_weight * g).sum().unsqueeze(dim=0)
        grad_weight = between * grad_weight
        return grad_weight, grad_alpha, None, None, None, None

class LSQPlusActivationQuantizer(nn.Module):
    def __init__(self, a_bits, all_positive=False,batch_init = 20):
        super(LSQPlusActivationQuantizer, self).__init__()
        self.a_bits = nn.Parameter(torch.tensor(a_bits), requires_grad=False)
        self.s_bits = 20
        self.n = 1 << 16
        self.all_positive = all_positive
        self.batch_init = batch_init
        if self.all_positive:
            # unsigned activation is quantized to [0, 2^b-1]
            self.Qn = 0
            self.Qp = 2 ** a_bits - 1
        else:
            # signed weight/activation is quantized to [-2^(b-1), 2^(b-1)-1]
            self.Qn = - 2 ** (a_bits - 1)
            self.Qp = 2 ** (a_bits - 1) - 1
        self.s = torch.nn.Parameter(torch.ones(1).squeeze(0), requires_grad=True)
        self.beta = torch.nn.Parameter(torch.ones(1).squeeze(0), requires_grad=True)
        self.g = torch.nn.Parameter(torch.ones(1).squeeze(0), requires_grad=True)
        self.register_buffer('init_state', torch.zeros(1))

    def forward(self, activation):
        if self.training:
            if self.init_state==0:
                self.g.data = torch.tensor(1.0/math.sqrt(activation.numel() * self.Qp))
                mina = torch.min(activation.detach())
                self.s.data = (torch.max(activation.detach()) - mina)/(self.Qp-self.Qn)
                self.beta.data = mina - self.s.data *self.Qn
                self.init_state += 1
            elif self.init_state<self.batch_init:
                mina = torch.min(activation.detach())
                self.s.data = self.s.data*0.9 + 0.1*(torch.max(activation.detach()) - mina)/(self.Qp-self.Qn)
                self.beta.data = self.s.data*0.9 + 0.1* (mina - self.s.data * self.Qn)
                self.init_state += 1
            elif self.init_state==self.batch_init:
                self.init_state += 1

        if self.a_bits.item() == 32:
            q_a = activation
        elif self.a_bits.item() == 1:
            print('！Binary quantization is not supported ！')
            assert self.a_bits.item() != 1
        else:
            q_a = ALSQPlus.apply(activation, self.s, self.beta, self.g, self.Qn, self.Qp)
        return q_a

class LSQPlusWeightQuantizer(nn.Module):
    def __init__(self, w_bits, all_positive=False, per_channel=False, batch_init = 20, shape=(1,), saved=True):
        super(LSQPlusWeightQuantizer, self).__init__()
        self.w_bits = nn.Parameter(torch.tensor(w_bits), requires_grad=False)
        self.s_bits = 20
        self.n = 1 << 16
        self.all_positive = all_positive
        self.batch_init = batch_init
        if self.all_positive:
            # unsigned activation is quantized to [0, 2^b-1]
            self.Qn = 0
            self.Qp = 2 ** w_bits - 1
        else:
            # signed weight/activation is quantized to [-2^(b-1), 2^(b-1)-1]
            self.Qn = - 2 ** (w_bits - 1)
            self.Qp = 2 ** (w_bits - 1) - 1
        self.per_channel = per_channel
        self.register_buffer('init_state', torch.zeros(1))
        if not per_channel:
           self.s = torch.nn.Parameter(torch.ones(1).squeeze(0), requires_grad=True)
        else:
            self.s = torch.nn.Parameter(torch.ones(shape[0]), requires_grad=True)

        self.saved = saved
        if saved:
            self.wq = torch.nn.Parameter(torch.ones(shape), requires_grad=True)
        self.g = torch.nn.Parameter(torch.ones(1).squeeze(0), requires_grad=True)
        
    def forward(self, weight):
        '''
        For this work, each layer of weights and each layer of activations has a distinct step size, represented
as an fp32 value, initialized to 2h|v|i/√OP , computed on either the initial weights values or the first
batch of activations, respectively
        '''
        if self.training:
            if self.init_state==0:
                self.div = 2**self.w_bits.item()-1
                self.g.data = torch.tensor(1.0/math.sqrt(weight.numel() * self.Qp))
                if self.per_channel:
                    # weight_tmp = weight.detach().contiguous().view(weight.size()[0], -1)
                    weight_tmp = weight.detach().view(weight.size()[0], -1)
                    mean = torch.mean(weight_tmp, dim=1)
                    std = torch.std(weight_tmp, dim=1)
                    v, _ = torch.max(torch.stack([torch.abs(mean-3*std), torch.abs(mean + 3*std)]), dim=0)
                    self.s.data = v/self.div
                else:
                    mean = torch.mean(weight.detach())
                    std = torch.std(weight.detach())
                    self.s.data = max([torch.abs(mean-3*std), torch.abs(mean + 3*std)])/self.div
                self.init_state += 1
            elif self.init_state<self.batch_init:
                self.div = 2**self.w_bits.item()-1
                if self.per_channel:
                    # weight_tmp = weight.detach().contiguous().view(weight.size()[0], -1)
                    weight_tmp = weight.detach().view(weight.size()[0], -1)
                    mean = torch.mean(weight_tmp, dim=1)
                    std = torch.std(weight_tmp, dim=1)
                    v, _ = torch.max(torch.stack([torch.abs(mean-3*std), torch.abs(mean + 3*std)]), dim=0)
                    self.s.data = v*0.9 + 0.1*v/self.div
                else:
                    mean = torch.mean(weight.detach())
                    std = torch.std(weight.detach())
                    self.s.data = self.s.data*0.9 + 0.1*max([torch.abs(mean-3*std), torch.abs(mean + 3*std)])/self.div
                self.init_state += 1
            elif self.init_state==self.batch_init:
                self.init_state += 1

        if self.w_bits.item() == 32:
            w_q = weight
        elif self.w_bits.item() == 1:
            print('！Binary quantization is not supported ！')
            assert self.w_bits.item() != 1
        else:
            if self.saved:
                w_q, self.wq.data = WLSQPlus.apply(weight, self.s, self.g, self.Qn, self.Qp, self.per_channel)
            else:
                w_q, _ = WLSQPlus.apply(weight, self.s, self.g, self.Qn, self.Qp, self.per_channel)

        return w_q

class QuantConv2d(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, 
                 w_bits=8, a_bits=8, all_positive=False, per_channel=False, batch_init=20):
        super(QuantConv2d, self).__init__(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias)
        
        # Weight quantizer
        shape = self.weight.shape
        self.weight_quantizer = LSQPlusWeightQuantizer(w_bits, all_positive, per_channel, batch_init, shape)
        
        # Activation quantizer
        self.act_quantizer = LSQPlusActivationQuantizer(a_bits, all_positive, batch_init)
    
    def forward(self, x):
        # Quantize input activations
        x_q = self.act_quantizer(x)
        
        # Quantize weights
        w_q = self.weight_quantizer(self.weight)
        
        # Use quantized values for convolution
        output = F.conv2d(
            x_q, w_q, self.bias, self.stride, self.padding, self.dilation, self.groups
        )
        
        return output


class QuantLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True, 
                 w_bits=8, a_bits=8, all_positive=False, per_channel=False, batch_init=20):
        super(QuantLinear, self).__init__(in_features, out_features, bias)
        
        # Weight quantizer
        shape = self.weight.shape
        self.weight_quantizer = LSQPlusWeightQuantizer(w_bits, all_positive, per_channel, batch_init, shape)
        
        # Activation quantizer
        self.act_quantizer = LSQPlusActivationQuantizer(a_bits, all_positive, batch_init)
    
    def forward(self, x):
        # Quantize input activations
        x_q = self.act_quantizer(x)
        
        # Quantize weights
        w_q = self.weight_quantizer(self.weight)
        
        # Use quantized values for linear operation
        output = F.linear(x_q, w_q, self.bias)
        
        return output


def prepare(model, w_bits=8, a_bits=8, all_positive=False, per_channel=False, batch_init=20):
    """
    Replaces Conv2d and Linear layers in the model with QuantConv2d and QuantLinear layers.
    
    Args:
        model: The model to be quantized
        w_bits: Number of bits for weight quantization
        a_bits: Number of bits for activation quantization
        all_positive: Whether the quantization is all positive
        per_channel: Whether to use per-channel quantization for weights
        batch_init: Number of batches for quantizer initialization
        
    Returns:
        Quantized model with replaced layers
    """
    quant_model = copy.deepcopy(model)
    
    for name, module in list(quant_model.named_children()):
        # If it's a container (like Sequential), recursively apply to its children
        if len(list(module.children())) > 0:
            setattr(quant_model, name, prepare(module, w_bits, a_bits, all_positive, per_channel, batch_init))
        
        # If it's a Conv2d layer, replace it with QuantConv2d
        elif isinstance(module, nn.Conv2d):
            quant_conv = QuantConv2d(
                module.in_channels, 
                module.out_channels,
                module.kernel_size,
                module.stride,
                module.padding,
                module.dilation,
                module.groups,
                module.bias is not None,
                w_bits,
                a_bits,
                all_positive,
                per_channel,
                batch_init
            )
            
            # Copy weights and bias if exists
            quant_conv.weight.data = module.weight.data.clone()
            if module.bias is not None:
                quant_conv.bias.data = module.bias.data.clone()
                
            setattr(quant_model, name, quant_conv)
            
        # If it's a Linear layer, replace it with QuantLinear
        elif isinstance(module, nn.Linear):
            quant_linear = QuantLinear(
                module.in_features,
                module.out_features,
                module.bias is not None,
                w_bits,
                a_bits,
                all_positive,
                per_channel,
                batch_init
            )
            
            # Copy weights and bias if exists
            quant_linear.weight.data = module.weight.data.clone()
            if module.bias is not None:
                quant_linear.bias.data = module.bias.data.clone()
                
            setattr(quant_model, name, quant_linear)
    
    return quant_model

## Q3- Quantize base module

In [ ]:
def plot_acc_loss():
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs+1), train_accs, label='Train Acc')
    plt.plot(range(1, epochs+1), test_accs, label='Test Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.legend(); plt.title('Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
    plt.plot(range(1, epochs+1), test_losses, label='Test Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.title('Loss')
    
    plt.tight_layout()
    plt.show()

In [ ]:
model = copy.deepcopy(baseline_model)
model = prepare(model, w_bits=8, a_bits=8).to(device)

epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

In [ ]:
train_accs, test_accs, train_losses, test_losses = [], [], [], []

train(epochs)

acc = test(model)
print(f'quantize Accuracy: {acc:.2f}%')
q8_model = model

plot_acc_loss()

In [ ]:
print(f"model size: {size_model} params | {size_model * 8 / 8e6:.2f} MB")

دقت مدل به اندازه مدل اولیه است و کاهش دقت نداشتیم

## For 6, 4 bit

In [ ]:
model = copy.deepcopy(baseline_model)
model = prepare(model, w_bits=6, a_bits=6).to(device)

epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

In [ ]:
train_accs, test_accs, train_losses, test_losses = [], [], [], []

train(epochs)

acc = test(model)
print(f'quantize Accuracy: {acc:.2f}%')
q6_model = model

plot_acc_loss()

In [ ]:
print(f"model size: {size_model} params | {size_model * 6 / 8e6:.2f} MB")

In [ ]:
model = copy.deepcopy(baseline_model)
model = prepare(model, w_bits=4, a_bits=4).to(device)

epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

In [ ]:
train_accs, test_accs, train_losses, test_losses = [], [], [], []

train(epochs)

acc = test(model)
print(f'quantize Accuracy: {acc:.2f}%')
q4_model = model

plot_acc_loss()

In [ ]:
print(f"model size: {size_model} params | {size_model * 4 / 8e6:.2f} MB")

در ۴ بیت ۰.۶ درصد افت دقت و در ۶ بیت ۰.۲۵ درصد افت دقت داشتیم

## Best prune model

In [ ]:
# l1_uns_glob_step : 0.8, 0.6, 0.4

for l1 in [0.8, 0.6, 0.4]:
    for q_bit in [8, 6]:
        s_model = size_model * (1-l1)
        print(f"L1-Global Iterative Prune with {l1*100:.0f}% and quantize with {q_bit}bits size: {s_model:.0f} params | {s_model * q_bit / 8e6:.2f} MB")

In [ ]:
epochs = 8
# first for l1 % and second for q level
q_l1_uns_glob_step = dict()
for l1 in [0.8, 0.6, 0.4]:
    q_l1_uns_glob_step[l1] = dict()
    for q_bit in [8, 6]:
        model = copy.deepcopy(l1_uns_glob_step[l1])
        model = prepare(model, w_bits=q_bit, a_bits=q_bit).to(device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        train_accs, test_accs, train_losses, test_losses = [], [], [], []
        
        train(epochs)
        
        acc = test(model)
        print(f'L1-Global Iterative Prune with {l1*100:.0f}% and quantize with {q_bit}bits Accuracy: {acc:.2f}%')
        q_l1_uns_glob_step[l1][q_bit] = copy.deepcopy(model)
        
        plot_acc_loss()

در ابتدا دیدیم مدل

L1-Global Iterative

از همه بهتر عمل کرده بود
با مقدادیر مختلف آن مدل را کوانتایز کردیم و آموزش دادیم و با توجه به مقادیر به نظر بیشترین فشرده سازی یعنی ۸۰ درصد هرس و ۶ بیت دقت ۸۶.۹ را می دهد که تنها ۰.۷ کاهش دقت دارد ولی حجم آن به ۰.۳۶ مگابایت می رسد.